This notebook will be sketching out my implementation of the Resnet-18 network used in the [paper](http://cs231n.stanford.edu/reports/2017/pdfs/406.pdf) mentioned in initial_work.ipynb. I want to try building it out with the base components built into PyTorch and compare that to PyTorch's built-in Resnet constructors.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import common_utils


class ResBlock(nn.Module):
    """
    Class representing one two-layer residual learning block, as outlined in the paper
    "Deep Residual Learning for Image Recognition"
    """
    def __init__(self, in_dimension, dimension, stride=1):
        super(ResBlock, self).__init__()
        self.dimension = dimension
        self.in_dimension = in_dimension
        self.in_conv = nn.Conv2d(in_dimension, dimension, 3, stride=stride, padding=1)
        self.bnorm = nn.BatchNorm2d(dimension)
        self.conv = nn.Conv2d(dimension, dimension, 3, stride=1, padding=1)
        
    def forward(self, x):
        """
        Connectivity follows the diagram from the paper 'Deep Residual Learning for Image Recognition'
        """
        res = x
        out = F.relu(self.bnorm(self.in_conv(x)), inplace=True)
        out = self.bnorm(self.conv(out))
        # adding the residual weights to the layer output
        # need to downsample the residual in first layer of blocks with nonzero stride
        if (res.size() != out.size()):
            conv = nn.Conv2d(self.in_dimension, self.dimension, 1, stride=2)
            bnorm = nn.BatchNorm2d(self.dimension)
            res = bnorm(conv(res))
        out += res
        out = F.relu(out)
        
        return out

class HomegrownResnet18(nn.Module):
    
    def __init__(self, num_classes):
        super(HomegrownResnet18, self).__init__()
        self.in_conv = nn.Conv2d(3, 64, 7, stride=2, padding=3)
        self.bnorm = nn.BatchNorm2d(64)
        self.max_pool = nn.MaxPool2d(3,stride=2, padding = 1)
        self.chunk64 = nn.Sequential(*[ResBlock(64,64),ResBlock(64,64)])
        self.chunk128 = nn.Sequential(*[ResBlock(64,128,stride=2),ResBlock(128,128)])
        self.chunk256 = nn.Sequential(*[ResBlock(128,256,stride=2),ResBlock(256,256)])
        self.chunk512 = nn.Sequential(*[ResBlock(256,512,stride=2),ResBlock(512,512)])
        self.avgpool = nn.AvgPool2d(7,stride=2)
        self.fc = nn.Linear(512,num_classes)
        
        # initialize weights in a way that makes sense for ReLU layers
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = F.relu(self.bnorm(self.in_conv(x)),inplace=True)
        x = self.max_pool(x)
        x = self.chunk64(x)
        x = self.chunk128(x)
        x = self.chunk256(x)
        x = self.chunk512(x)
        
        x = self.avgpool(x)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        
        return x
        
net = HomegrownResnet18(38)
    
print(net)

HomegrownResnet18(
  (in_conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (chunk64): Sequential(
    (0): ResBlock(
      (in_conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ResBlock(
      (in_conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (chunk128): Sequential(
    (0): ResBlock(
      (in_conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))

In [2]:
train, test, val = common_utils.get_dataloaders()

In [ ]:
import torch.optim as optim
import matplotlib.pyplot as plt

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.001)

val_acc = []
iterations = []
train_len = len(train)

for epoch in range(30):
    
    running_loss = 0
    
    for i, sample in enumerate(train):
        images, labels = sample['images'], sample['labels']
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 80 == 79:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 80))
            running_loss = 0.0
            with torch.no_grad():
                total = 0
                correct = 0
                for sample in val:
                    images,labels = sample['images'], sample['labels']
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                val_acc.append(100*correct/total)
                iterations.append((i+1) + (train_len * epoch))
                print("[%d, %5d] test accuracy: %.3f" % (epoch + 1, i + 1, 100*correct/total))
                    
print('done')


[1,    80] loss: 3.498
[1,    80] test accuracy: 6.786
[1,   160] loss: 3.384
[1,   160] test accuracy: 10.143
[2,    80] loss: 3.337
[2,    80] test accuracy: 12.643
[2,   160] loss: 3.233
[2,   160] test accuracy: 11.786
[3,    80] loss: 3.240
[3,    80] test accuracy: 12.643
[3,   160] loss: 3.183
[3,   160] test accuracy: 14.929
[4,    80] loss: 3.169
[4,    80] test accuracy: 15.571
[4,   160] loss: 3.144
[4,   160] test accuracy: 15.000
[5,    80] loss: 3.138
[5,    80] test accuracy: 17.143
[5,   160] loss: 3.082
[5,   160] test accuracy: 15.571
[6,    80] loss: 3.068
[6,    80] test accuracy: 16.429
[6,   160] loss: 3.061
[6,   160] test accuracy: 14.571
[7,    80] loss: 3.015
[7,    80] test accuracy: 18.357
[7,   160] loss: 2.991
[7,   160] test accuracy: 18.786
[8,    80] loss: 2.984
[8,    80] test accuracy: 20.286
[8,   160] loss: 2.915
[8,   160] test accuracy: 20.571
[9,    80] loss: 2.892
[9,    80] test accuracy: 22.000
[9,   160] loss: 2.865
[9,   160] test accuracy: 

In [ ]:
plt.plot(iterations, val_acc)
plt.xlabel('number of iterations')
plt.ylabel('percent accuracy')
plt.show()